In [1]:
import transformers
import torch.nn as nn


DEVICE = "cpu"
MAX_LEN = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
BERT_PATH = '../data/bert'
MODEL_PATH = "bert_model.bin"
TRAINING_FILE = "bert_train.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)


In [2]:
import pandas as pd
import tensorflow as tf

SENTENCE_EMOTIONS_TEST_FILE = '../data/sentence_emotions_test.pickle' 
SENTENCE_EMOTIONS_TRAIN_FILE = '../data/sentence_emotions_train.pickle'
CONTINUES_TO_BINARY_THRESHOLD = 0.5

from limbic.limbic_constants import AFFECT_INTENSITY_EMOTIONS as EMOTIONS


def load_data_file(file_path):
    data = pd.read_pickle(file_path)
    
    data_sentences = data['text'] #.str.lower().apply(lambda x: utils.preprocess_sentence(x))
    y_data = data[EMOTIONS].values

    #     # This will be used throughout the notebook to compute performance 
#     y_data_labeled = utils.continuous_labels_to_binary(y_data, CONTINUES_TO_BINARY_THRESHOLD)   

#     # This representation will be needed for sklearn later in this notebook. 
#     x_data = tokenizer.texts_to_sequences(data_sentences)
#     x_data = tf.keras.preprocessing.sequence.pad_sequences(x_data, maxlen=MAX_LEN)

#     return data, x_data, y_data, y_data_labeled, data_sentences
    return data, y_data, data_sentences


train, y_train_labeled, train_sentences = load_data_file(SENTENCE_EMOTIONS_TRAIN_FILE)
# test, x_test, y_test, y_test_labeled, test_sentences = load_data_file(SENTENCE_EMOTIONS_TEST_FILE)

print(f'train shape: {train.shape}')
# print(f'test shape: {test.shape}')

train shape: (76340, 5)


In [3]:
train[EMOTIONS].values

array([[0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.406, 0.206],
       ...,
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ]])

In [4]:
import torch


class BERTDataset:
    def __init__(self, text, target):
        self.text = text
        self.target = target
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        text = str(self.text[item])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
        )
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.float),
        }


In [5]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, NUM_LABELS)

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo = self.bert_drop(o2)
        output = nn.functional.softmax(self.out(bo), dim=1)
        return output


In [6]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

from torch.utils.tensorboard import SummaryWriter

# Tensorboard + PyTorch: https://towardsdatascience.com/a-complete-guide-to-using-tensorboard-with-pytorch-53cb2301e8c3
TB_WRITER = SummaryWriter()

def loss_fn(outputs, targets, num_labels):
    return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, num_labels))


def train_fn(data_loader, model, optimizer, device, scheduler, num_labels, epoch):
    model.train()

    total_loss = 0
    n = 0
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d['ids']
        token_type_ids = d['token_type_ids']
        mask = d['mask']
        targets = d['targets']

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)

        loss = loss_fn(outputs, targets, num_labels)
        total_loss += loss
        n += 1

        loss.backward()
        optimizer.step()
        scheduler.step()
        
    TB_WRITER.add_scalar("Loss/train", total_loss/n, epoch)

        
def eval_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
#             fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [7]:
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

NUM_LABELS = 4


def run():
#     dfx = pd.read_csv(TRAINING_FILE).fillna("none")
#     dfx.sentiment = dfx.sentiment.apply(lambda x: 1 if x == "positive" else 0)
    dfx, y_train_labeled, train_sentences = load_data_file(SENTENCE_EMOTIONS_TRAIN_FILE)

    df_train, df_valid = model_selection.train_test_split(
        dfx.sample(n=100, random_state=1), test_size=0.1, random_state=42, #stratify=dfx.sentiment.values # Consider iterative stratified
    )

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    train_dataset = BERTDataset(
        text=df_train.text.values, target=df_train[EMOTIONS].values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4
    )

    valid_dataset = BERTDataset(
        text=df_valid.text.values, target=df_valid[EMOTIONS].values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
    )

    device = torch.device(DEVICE)
    model = BERTBaseUncased()
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
    )

    best_accuracy = 0
    for epoch in tqdm(range(EPOCHS), 'epochs'):
        train_fn(train_data_loader, model, optimizer, device, scheduler, NUM_LABELS, epoch)
        outputs, targets = eval_fn(valid_data_loader, model, device)
        outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Accuracy Score = {accuracy}")
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = accuracy


In [8]:
# run()

In [9]:
from sklearn.metrics import label_ranking_average_precision_score

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.ERROR)

print('training....')

dfx, y_train_labeled, train_sentences = load_data_file(SENTENCE_EMOTIONS_TRAIN_FILE)

df_train, df_valid = model_selection.train_test_split(
    dfx.sample(n=100, random_state=1), test_size=0.1, random_state=42, #stratify=dfx.sentiment.values # Consider iterative stratified
)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

train_dataset = BERTDataset(
    text=df_train.text.values, target=df_train[EMOTIONS].values
)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4
)

valid_dataset = BERTDataset(
    text=df_valid.text.values, target=df_valid[EMOTIONS].values
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
)

device = torch.device(DEVICE)
model = BERTBaseUncased()
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]

num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
)

best_accuracy = 0
for epoch in tqdm(range(EPOCHS), 'epochs'):
    train_fn(train_data_loader, model, optimizer, device, scheduler, NUM_LABELS, epoch)
    outputs, targets = eval_fn(valid_data_loader, model, device)
    targets = np.array(np.array(targets) >= 0.1).astype(int)
    outputs = np.array(outputs) #>= 0.5
    accuracy = label_ranking_average_precision_score(targets, outputs)
    print(f"Ranking Avg Precision Score = {accuracy}")
    if accuracy > best_accuracy:
        torch.save(model.state_dict(), MODEL_PATH)
        best_accuracy = accuracy

TB_WRITER.close()


training....


epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Ranking Avg Precision Score = 0.7916666666666666


...

  0%|          | 0/3 [00:00<?, ?it/s]

Ranking Avg Precision Score = 0.9083333333333334


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Ranking Avg Precision Score = 0.9083333333333334


In [10]:
# targets

In [11]:

# np.array(targets)
# label_ranking_average_precision_score(np.array(targets).as, np.array(outputs))



In [12]:
# y_true = np.array([[0.0, 1.0, 0.0, 0.0],
#  [0, 1, 0, 0]])
# y_score = np.array([[0.5 , 0.5, 0.5, 0.5],
#        [0.5, 0.5, 0.5, 0.5]])
# label_ranking_average_precision_score(y_true, y_score)


In [13]:
# t = np.array(targets) >= 0.5
# t.astype(int)

In [14]:
# np.array(np.array(targets) >= 0.1).astype(int)

In [15]:
PREDICTION_DICT = dict()
import time


def sentence_prediction(sentence):
    tokenizer = TOKENIZER
    max_len = MAX_LEN
    review = str(sentence)
    review = " ".join(review.split())

    inputs = tokenizer.encode_plus(
        review, None, add_special_tokens=True, max_length=max_len)

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    padding_length = max_len - len(ids)
    ids = ids + ([0] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)

    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)
    mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)

    ids = ids.to(DEVICE, dtype=torch.long)
    token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
    mask = mask.to(DEVICE, dtype=torch.long)

    outputs = MODEL(ids=ids, mask=mask, token_type_ids=token_type_ids)
    return outputs
#     outputs = torch.sigmoid(outputs).cpu().detach().numpy()
#     return outputs[0][0]


def predict(sentence):
    start_time = time.time()
    positive_prediction = sentence_prediction(sentence)

    response = {}
    response["response"] = {
        "prediction": positive_prediction,
        "sentence": str(sentence),
        "time_taken": str(time.time() - start_time),
    }
    return response

MODEL = BERTBaseUncased()
MODEL.load_state_dict(torch.load(MODEL_PATH))
MODEL.to(DEVICE)
MODEL.eval()



BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [16]:
# output = nn.functional.softmax(self.out(bo), dim=1)
p = predict('this is a sentence with hate')
p 

{'response': {'prediction': tensor([[0.1787, 0.5970, 0.1025, 0.1218]], grad_fn=<SoftmaxBackward>),
  'sentence': 'this is a sentence with hate',
  'time_taken': '0.24013590812683105'}}

In [17]:
p['response']['prediction'].detach().numpy()[0]

array([0.17873068, 0.5970073 , 0.10250984, 0.1217522 ], dtype=float32)

In [18]:
EMOTIONS

['sadness', 'joy', 'fear', 'anger']

In [19]:
from limbic.emotion.models.tf_limbic_model import utils

import pickle

SENTENCE_EMOTIONS_TRAIN_FILE = '../data/sentence_emotions_train.pickle'
CONTINUES_TO_BINARY_THRESHOLD = 0.5

VERSION = '2019-11-16'

metadata_file = f'model_metadata_{VERSION}.txt'
tokenizer_file = f'tokenizer_{VERSION}.pickle'

with open(tokenizer_file, 'rb') as tokenizer_f:
    tokenizer = pickle.load(tokenizer_f)


def load_data_file(file_path):
    data = pd.read_pickle(file_path)
    data_sentences = data['text'].str.lower().apply(lambda x: utils.preprocess_sentence(x))
    y_data = data[EMOTIONS].values
    # This will be used throughout the notebook to compute performance 
    y_data_labeled = utils.continuous_labels_to_binary(y_data, CONTINUES_TO_BINARY_THRESHOLD)   

    # This representation will be needed for sklearn later in this notebook. 
    x_data = tokenizer.texts_to_sequences(data_sentences)
    x_data = tf.keras.preprocessing.sequence.pad_sequences(x_data, maxlen=MAX_LEN)
    
    return data, x_data, y_data, y_data_labeled, data_sentences

test, x_test, y_test, y_test_labeled, test_sentences = load_data_file(SENTENCE_EMOTIONS_TEST_FILE)


In [20]:
y_pred_bert = []
for idx, sentence in tqdm(enumerate(test_sentences[:20])):
    print('*' * 10)
    print(sentence)
    print(y_test_labeled[idx])
    p = predict(sentence)
    print(p['response']['prediction'].detach().numpy()[0])
    y_pred_bert.append(p['response']['prediction'].detach().numpy()[0])
    

|          | 0/? [00:00<?, ?it/s]

**********
but i am sure i have always thought of christmas time when it has come roundapart from the veneration due to its sacred name and origin if anything belonging to it can be apart from thatas a good time a kind forgiving charitable pleasant time the only time i know of in the long calendar of the year when men and women seem by one consent to open their shutup hearts freely and to think of people below them as if they really were fellowpassengers to the grave and not another race of creatures bound on other journeys
[1 1 1 0]
[0.13968161 0.7154621  0.09387286 0.05098342]
**********
without substantially accomplishing this part of their undertaking they would have very imperfectly fulfilled the object of their appointment or the expectation of the public yet that it could not be easily accomplished will be denied by no one who is unwilling to betray his ignorance of the subject
[1 1 0 1]
[0.18566225 0.37759364 0.38168907 0.05505502]
**********
they early introduce us to and deta

In [21]:
import numpy as np
from sklearn.metrics import classification_report


y_pred_bert_labeled = utils.continuous_labels_to_binary(np.array([list(x) for x in y_pred_bert]), 0.5)

print(classification_report(y_test_labeled[:20], y_pred_bert_labeled, target_names=EMOTIONS))
    

              precision    recall  f1-score   support

     sadness       0.00      0.00      0.00         3
         joy       0.18      0.50      0.27         4
        fear       0.00      0.00      0.00         1
       anger       0.00      0.00      0.00         1

   micro avg       0.17      0.22      0.19         9
   macro avg       0.05      0.12      0.07         9
weighted avg       0.08      0.22      0.12         9
 samples avg       0.10      0.07      0.07         9

